In [1]:
from simulation_utils import * 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import gpboost as gpb
#import pdpbox
#from pdpbox import pdp
sns.set()

# Initial experiments - Small data setting (only 10 observations per group)

## Non-linear function; Shared GP in data generating process

In [2]:
# Generate data
n, m = 500, 50  # Number of observations and groups
p = int(n/m) # Number of observations per group
n_datasets = 20 
n_valid = 5
datasets, validation_datasets = generate_datasets(n, m, p, n_datasets, n_valid, func='make_friedman3', random_state=60, shared_gp=True)

# Create groups
groups = np.arange(n)
for i in range(m):
    groups[i*p:(i+1)*p] = i

# Create times for validation datasets
times = np.tile(np.arange(p)*10+10, m)

## GPBoost hyperparameter tuning

### Random Intercept model

In [3]:
# Random slope
gp_model = gpb.GPModel(group_data=np.column_stack((groups, times)))
# Candidate parameter grid
param_grid = {'learning_rate': [0.01, 0.1, 0.5, 1], 
              'max_depth': [1, 2, 3, 4, 5, 10, 50],
              'min_data_in_leaf': [1, 10, 20, 50]}

# Other parameters not contained in the grid of tuning parameters
params = {'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

opt_params = grid_search_tune_parameters_multiple(param_grid=param_grid,
                                                  params=params,
                                                  nfold=5,
                                                  gp_model=gp_model,
                                                  use_gp_model_for_validation=True,
                                                  train_sets=validation_datasets['data_train'],
                                                  verbose_eval=0,
                                                  num_boost_round=1000, 
                                                  early_stopping_rounds=10,
                                                  shuffle=False,
                                                  seed=1,
                                                  metrics='rmse')

print('Best number of iterations: ', opt_params['best_iter'])
print('Best score: ', opt_params['best_score'])
print('Best parameters: ', opt_params['best_params'])

Starting deterministic grid search with 112 parameter combinations...


 |--------------------------------------------------| 0.8929% 



 |█-------------------------------------------------| 1.7857% 



 |█-------------------------------------------------| 2.6786% 



 |██------------------------------------------------| 3.5714% 



 |██------------------------------------------------| 4.4643% 



 |███-----------------------------------------------| 5.3571% 



 |███-----------------------------------------------| 6.2500% 



 |████----------------------------------------------| 7.1429% 



 |████----------------------------------------------| 8.0357% 



 |████----------------------------------------------| 8.9286% 



 |█████---------------------------------------------| 9.8214% 



 |█████---------------------------------------------| 10.7143% 



 |██████--------------------------------------------| 11.6071% 



 |██████--------------------------------------------| 12.5000% 



 

### GPBoost with independent GPs

In [4]:
# Independent GP
gp_model = gpb.GPModel(group_data=groups, gp_coords=times, cluster_ids=groups, cov_function='exponential', cov_fct_shape=2.5)

# Candidate parameter grid
param_grid = {'learning_rate': [0.01, 0.1, 0.5, 1], 
              'max_depth': [1, 2, 3, 4, 5, 10, 50],
              'min_data_in_leaf': [1, 10, 20, 50]}

# Other parameters not contained in the grid of tuning parameters
params = {'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

opt_params = grid_search_tune_parameters_multiple(param_grid=param_grid,
                                                  params=params,
                                                  nfold=5,
                                                  gp_model=gp_model,
                                                  use_gp_model_for_validation=True,
                                                  train_sets=validation_datasets['data_train'],
                                                  verbose_eval=1,
                                                  num_boost_round=1000, 
                                                  early_stopping_rounds=10,
                                                  shuffle=False,
                                                  seed=1,
                                                  metrics='rmse')
                                                  
print('Best number of iterations: ', opt_params['best_iter'])
print('Best score: ', opt_params['best_score'])
print('Best parameters: ', opt_params['best_params'])

Starting deterministic grid search with 112 parameter combinations...


 |--------------------------------------------------| 0.8929% 

Trying parameter combination 1 of 112: {'learning_rate': 0.01, 'max_depth': 1, 'min_data_in_leaf': 1} ...
***** New best score (1.839544647756608) found for the following parameter combination:
{'learning_rate': 0.01, 'max_depth': 1, 'min_data_in_leaf': 1, 'num_boost_round': 998.8}


 |█-------------------------------------------------| 1.7857% 

Trying parameter combination 2 of 112: {'learning_rate': 0.1, 'max_depth': 1, 'min_data_in_leaf': 1} ...
***** New best score (1.822517987860015) found for the following parameter combination:
{'learning_rate': 0.1, 'max_depth': 1, 'min_data_in_leaf': 1, 'num_boost_round': 188.8}


 |█-------------------------------------------------| 2.6786% 

Trying parameter combination 3 of 112: {'learning_rate': 0.5, 'max_depth': 1, 'min_data_in_leaf': 1} ...
***** New best score (1.8214144157542655) found for the followi

### GPBoost with shared GPs

In [6]:
# Shared GP
gp_model = gpb.GPModel(group_data=groups, gp_coords=times, cov_function='exponential', cov_fct_shape=2.5)

# Use the same hyperparameters as for Independent GP and only cross-validate to find num_boost_round
params = {'learning_rate': 0.1,
          'max_depth': 3,
          'min_data_in_leaf': 10,
          'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

opt_params = grid_search_tune_parameters_multiple(param_grid={},
                                                  params=params,
                                                  nfold=5,
                                                  gp_model=gp_model,
                                                  use_gp_model_for_validation=True,
                                                  train_sets=validation_datasets['data_train'],
                                                  verbose_eval=1,
                                                  num_boost_round=1000, 
                                                  early_stopping_rounds=10,
                                                  shuffle=False,
                                                  seed=1,
                                                  metrics='rmse')
                                                  
print('Best number of iterations: ', opt_params['best_iter'])
print('Best score: ', opt_params['best_score'])
print('Best parameters: ', opt_params['best_params'])

Starting deterministic grid search with 1 parameter combinations...


 |██████████████████████████████████████████████████| 100.0000% 


Trying parameter combination 1 of 1: {} ...
***** New best score (1.5869322061031315) found for the following parameter combination:
{'num_boost_round': 57.0}
Best number of iterations:  57.0
Best score:  1.5869322061031315
Best parameters:  {}


### Gradient-boosted trees including groups as a categorical variable

In [5]:
# GBT with grouping variable as a categorical feature
datasets_cat, validation_datasets_cat = generate_datasets(n, m, p, n_datasets, n_valid, func='make_friedman3', 
                                                            shared_gp=True, random_state=60, include_cat_feature=True)

# Candidate parameter grid
param_grid = {'num_leaves': [10, 20, 50, 200, 500, 1000, 5000, 2**10], 
              'max_depth': [1, 2, 3, 4, 5, 10, 50],
              'min_data_in_leaf': [1, 5, 10, 20, 30, 50]}

# Other parameters not contained in the grid of tuning parameters
params = {'objective': 'regression_l2',
          'verbose': 0,
          'learning_rate' : 0.10}
          #'device_type' : 'gpu'}

opt_params = grid_search_tune_parameters_multiple(param_grid=param_grid,
                                                  params=params,
                                                  nfold=5,
                                                  gp_model=None,
                                                  use_gp_model_for_validation=True,
                                                  train_sets=validation_datasets_cat['data_train'],
                                                  verbose_eval=1,
                                                  num_boost_round=1000, 
                                                  early_stopping_rounds=10,
                                                  shuffle=False,
                                                  seed=1,
                                                  metrics='rmse')
                                                  
print('Best number of iterations: ', opt_params['best_iter'])
print('Best score: ', opt_params['best_score'])
print('Best parameters: ', opt_params['best_params'])

Starting deterministic grid search with 336 parameter combinations...


 |--------------------------------------------------| 0.2976% 

Trying parameter combination 1 of 336: {'num_leaves': 10, 'max_depth': 1, 'min_data_in_leaf': 1} ...


/usr/local/lib/python3.7/dist-packages/gpboost/basic.py:1822: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/usr/local/lib/python3.7/dist-packages/gpboost/basic.py:1339: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


***** New best score (1.6344592188159859) found for the following parameter combination:
{'num_leaves': 10, 'max_depth': 1, 'min_data_in_leaf': 1, 'num_boost_round': 182.0}


 |--------------------------------------------------| 0.5952% 

Trying parameter combination 2 of 336: {'num_leaves': 20, 'max_depth': 1, 'min_data_in_leaf': 1} ...


 |--------------------------------------------------| 0.8929% 

Trying parameter combination 3 of 336: {'num_leaves': 50, 'max_depth': 1, 'min_data_in_leaf': 1} ...


 |█-------------------------------------------------| 1.1905% 

Trying parameter combination 4 of 336: {'num_leaves': 200, 'max_depth': 1, 'min_data_in_leaf': 1} ...


 |█-------------------------------------------------| 1.4881% 

Trying parameter combination 5 of 336: {'num_leaves': 500, 'max_depth': 1, 'min_data_in_leaf': 1} ...


 |█-------------------------------------------------| 1.7857% 

Trying parameter combination 6 of 336: {'num_leaves': 1000, 'max_depth': 1, 'min_data_in_le

In [3]:
# Create results dataframe
results_1 = pd.DataFrame(columns=["RMSE Interpolation (mean)", "RMSE Interpolation (std)", "RMSE Extrapolation (mean)", "RMSE Extrapolation (std)", "RMSE_F (mean)", "RMSE_F (std)", "Time"],
                         index=['Linear Mixed Effects Model with no fixed features (random intercept)',
                                   'Linear Mixed Effects Model with Random Intercept',
                                   'Linear Mixed Effects Model with Shared Gaussian Process',
                                   'Linear Mixed Effects Model with Independent Gaussian Process',
                                   'Gradient-boosted tree with group as categorical variable (no random effects)',
                                   'GPBoost with Random Intercept',
                                   'GPBoost with Shared Gaussian Process',
                                   'GPBoost with Independent Gaussian Process'])

In [8]:
# 1. Linear Mixed Effects Model with no fixed features (random intercept)
time_list, RMSE_list1, RMSE_list2 = train_and_test(datasets, n_datasets, merf=False, linear=True, GP=False, no_features=True)

results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "Time"] = np.mean(time_list)
results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "RMSE Interpolation (std)"] = np.std(RMSE_list2)

  0%|          | 0/20 [00:00<?, ?it/s]

In [17]:
# 2. Linear Mixed Effects Model with Random Intercept
time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, merf=False, linear=True, GP=False)

results_1.loc["Linear Mixed Effects Model with Random Intercept", "Time"] = np.mean(time_list)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [5]:
# 3. Linear Mixed Effects Model with Shared Gaussian Process
mod_name = 'Linear Mixed Effects Model with Shared Gaussian Process'
time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, merf=False, linear=True, GP=True, shared=True, kernel='exponential', matern_param=2.5)

results_1.loc[mod_name, "Time"] = np.mean(time_list)
results_1.loc[mod_name, "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc[mod_name, "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc[mod_name, "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc[mod_name, "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc[mod_name, "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc[mod_name, "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [6]:
# 4. Linear Mixed Effects Model with Individual Gaussian Process
mod_name = 'Linear Mixed Effects Model with Independent Gaussian Process'
time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, merf=False, linear=True, GP=True, shared=False, kernel='exponential', matern_param=2.5)

results_1.loc[mod_name, "Time"] = np.mean(time_list)
results_1.loc[mod_name, "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc[mod_name, "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc[mod_name, "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc[mod_name, "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc[mod_name, "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc[mod_name, "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [12]:
# 5. Gradient tree-boosting including the grouping variable as a categorical variable ('Boosting_Cat')
datasets_cat, validation_datasets_cat = generate_datasets(n, m, p, n_datasets, n_valid, func='make_friedman3', 
                                                          shared_gp=True, random_state=60, include_cat_feature=True)

params = {'num_leaves': 10, 'max_depth': 4, 'min_data_in_leaf': 20}
time_list, RMSE_list1, RMSE_list2, _ = train_and_test(datasets_cat, n_datasets, merf=False, GPBoost_cat=True, linear=False, 
                                                      GP=False, num_boost_round=69, params=params)

results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "Time"] = np.mean(time_list)
results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "RMSE Interpolation (std)"] = np.std(RMSE_list2) 

  0%|          | 0/20 [00:00<?, ?it/s]

[GPBoost] [Info] Total Bins 536
[GPBoost] [Info] Number of data points in the train set: 300, number of used features: 6
[GPBoost] [Info] Start training from score 5.353720
Error encountered
[GPBoost] [Info] Total Bins 535
[GPBoost] [Info] Number of data points in the train set: 300, number of used features: 6
[GPBoost] [Info] Start training from score 5.787741
Error encountered
[GPBoost] [Info] Total Bins 535
[GPBoost] [Info] Number of data points in the train set: 300, number of used features: 6
[GPBoost] [Info] Start training from score 5.279032
Error encountered
[GPBoost] [Info] Total Bins 535
[GPBoost] [Info] Number of data points in the train set: 300, number of used features: 6
[GPBoost] [Info] Start training from score 5.574934
Error encountered
[GPBoost] [Info] Total Bins 536
[GPBoost] [Info] Number of data points in the train set: 300, number of used features: 6
[GPBoost] [Info] Start training from score 4.888584
Error encountered
[GPBoost] [Info] Total Bins 536
[GPBoost] [In

In [18]:
# 6. GPBoost with Random Intercept
params = {'learning_rate': 0.1, 'max_depth': 2, 'min_data_in_leaf': 20, 
            'objective': 'regression_l2',
            'verbose': 0,
            'num_leaves': 2**10}
          
time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, num_boost_round=110, params=params, merf=False, linear=False, GP=False)
results_1.loc["GPBoost with Random Intercept", "Time"] = np.mean(time_list)
results_1.loc["GPBoost with Random Intercept", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["GPBoost with Random Intercept", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["GPBoost with Random Intercept", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["GPBoost with Random Intercept", "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc["GPBoost with Random Intercept", "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc["GPBoost with Random Intercept", "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [21]:
# 7. GPBoost with Shared Gaussian Process
params = {'learning_rate': 0.1,
          'max_depth': 3,
          'min_data_in_leaf': 10,
          'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, num_boost_round=57, params=params, merf=False, 
                                                            linear=False, GP=True, shared=True, kernel='exponential', matern_param=2.5)  
results_1.loc["GPBoost with Shared Gaussian Process", "Time"] = np.mean(time_list)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [15]:
# 8. GPBoost with Independent Gaussian Process
params = {'learning_rate': 0.1,
          'max_depth': 3,
          'min_data_in_leaf': 10,
          'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, num_boost_round=59, params=params,
                                         merf=False, linear=False, GP=True, shared=False, kernel='exponential', matern_param=2.5)
results_1.loc["GPBoost with Independent Gaussian Process", "Time"] = np.mean(time_list)
results_1.loc["GPBoost with Independent Gaussian Process", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["GPBoost with Independent Gaussian Process", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["GPBoost with Independent Gaussian Process", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["GPBoost with Independent Gaussian Process", "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc["GPBoost with Independent Gaussian Process", "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc["GPBoost with Independent Gaussian Process", "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [23]:
results_1

,RMSE Interpolation (mean),RMSE Interpolation (std),RMSE Extrapolation (mean),RMSE Extrapolation (std),RMSE_F (mean),RMSE_F (std),Time
Linear Mixed Effects Model with no fixed features (random intercept),2.161386,0.144212,2.020361,0.141578,NaN,NaN,0.007116
Linear Mixed Effects Model with Random Intercept,2.025748,0.17179,1.903476,0.130406,1.883588,0.1397,0.006701
Linear Mixed Effects Model with Shared Gaussian Process,1.467404,0.102062,1.653817,0.154665,2.045899,0.193669,0.532024
Linear Mixed Effects Model with Independent Gaussian Process,1.865953,0.148183,1.928368,0.128406,1.877296,0.150635,0.059837
Gradient-boosted tree with group as categorical variable (no random effects),1.652942,0.12821,1.665051,0.153298,NaN,NaN,0.017372
GPBoost with Random Intercept,2.000748,0.162757,1.851331,0.117651,1.511642,0.139222,0.135921
GPBoost with Shared Gaussian Process,1.423728,0.083299,1.59164,0.167749,1.527835,0.140794,3.577772
GPBoost with Independent Gaussian Process,1.822165,0.138121,1.86279,0.141778,1.510909,0.148392,0.152636


In [17]:
results_1.to_csv('small_data_sharedGP_non_linear.csv')